In [1]:
!pip install tensorflow==2.8.1 tensorflow-gpu==2.8.1 tensorflow-hub opencv-python matplotlib

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [3]:
pip install protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [4]:
# if a GPU is being used
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [8]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [9]:
256*.75

192.0

In [18]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 192,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section (of keypoints)
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.3)
    
    cv2.imshow('Movenet Multipose', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [11]:
frame.shape

(480, 640, 3)

In [12]:
cap.release()
cv2.destroyAllWindows()

In [13]:
keypoints_with_scores

array([[[1.49755612e-01, 6.76890433e-01, 1.19283475e-01],
        [1.32329673e-01, 6.82676435e-01, 1.46953017e-01],
        [1.34862155e-01, 6.77708566e-01, 1.64429769e-01],
        [1.35009393e-01, 7.17829406e-01, 1.20958745e-01],
        [1.38559952e-01, 7.02708125e-01, 1.24155976e-01],
        [1.96711689e-01, 7.47380733e-01, 7.35130757e-02],
        [1.97318017e-01, 7.22569048e-01, 1.20765895e-01],
        [2.59481430e-01, 7.49312103e-01, 3.32527570e-02],
        [2.58124143e-01, 6.99302852e-01, 7.52483159e-02],
        [2.91500151e-01, 7.05196619e-01, 3.94176841e-02],
        [2.93348819e-01, 6.86605573e-01, 6.94786832e-02],
        [3.50484967e-01, 7.64394939e-01, 5.40227368e-02],
        [3.35594028e-01, 7.18117952e-01, 6.21687956e-02],
        [3.73149842e-01, 7.57774293e-01, 4.32561189e-02],
        [3.49406689e-01, 7.00738132e-01, 3.67677100e-02],
        [3.58572483e-01, 7.29993641e-01, 2.16550604e-02],
        [3.50135267e-01, 6.99815869e-01, 2.58685406e-02]],

       [[8.1

In [ ]:
Loop Through People

In [14]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Draw Keypoints


In [15]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

# Draw Edges

In [16]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [17]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)